# Resistance/Support AND Candles Patterns

In [1]:
import pandas as pd
import MetaTrader5 as mt5

In [2]:
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()
    
ativo = 'EURUSD'
df = pd.DataFrame(mt5.copy_rates_from_pos(ativo, mt5.TIMEFRAME_M5, 0, 500))
df['Data'] = pd.to_datetime(df['time'], unit='s').apply(lambda x: str(x))
df = df[['Data','open','high','low','close','real_volume']]
df.columns = ['Data','Open','High','Low','Close','Vol']
df = df.rename(columns = {'real_volume':'vol'})
df

,Data,Open,High,Low,Close,Vol
0,2023-01-12 06:55:00,1.07706,1.07736,1.07704,1.07716,0
1,2023-01-12 07:00:00,1.07718,1.07722,1.07705,1.07709,0
2,2023-01-12 07:05:00,1.07709,1.07709,1.07688,1.07698,0
3,2023-01-12 07:10:00,1.07701,1.07727,1.07700,1.07727,0
4,2023-01-12 07:15:00,1.07727,1.07751,1.07712,1.07739,0
...,...,...,...,...,...,...
495,2023-01-16 00:15:00,1.08290,1.08316,1.08287,1.08292,0
496,2023-01-16 00:20:00,1.08292,1.08294,1.08290,1.08294,0
497,2023-01-16 00:25:00,1.08294,1.08294,1.08291,1.08292,0
498,2023-01-16 00:30:00,1.08304,1.08325,1.08292,1.08321,0


# Support and Resistance FUNCTIONS

In [3]:
def support(df1, l, n1, n2): #n1 n2 before and after candle l
    for i in range(l-n1+1, l+1):
        if(df1.Low[i]>df1.Low[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df1.Low[i]<df1.Low[i-1]):
            return 0
    return 1

def resistance(df1, l, n1, n2): #n1 n2 before and after candle l
    for i in range(l-n1+1, l+1):
        if(df1.High[i]<df1.High[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df1.High[i]>df1.High[i-1]):
            return 0
    return 1

In [4]:
length = len(df)
High = list(df['High'])
Low = list(df['Low'])
Close = list(df['Close'])
Open = list(df['Open'])
bodydiff = [0] * length

Highdiff = [0] * length
Lowdiff = [0] * length
ratio1 = [0] * length
ratio2 = [0] * length

def isEngulfing(l):
    row=l
    bodydiff[row] = abs(Open[row]-Close[row])
    if bodydiff[row]<0.000001:
        bodydiff[row]=0.000001      

    bodydiffmin = 0.002
    if (bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        Open[row-1]<Close[row-1] and
        Open[row]>Close[row] and 
        (Open[row]-Close[row-1])>=-0e-5 and Close[row]<Open[row-1]): #+0e-5 -5e-5
        return 1

    elif(bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        Open[row-1]>Close[row-1] and
        Open[row]<Close[row] and 
        (Open[row]-Close[row-1])<=+0e-5 and Close[row]>Open[row-1]):#-0e-5 +5e-5
        return 2
    else:
        return 0
       
def isStar(l):
    bodydiffmin = 0.0020
    row=l
    Highdiff[row] = High[row]-max(Open[row],Close[row])
    Lowdiff[row] = min(Open[row],Close[row])-Low[row]
    bodydiff[row] = abs(Open[row]-Close[row])
    if bodydiff[row]<0.000001:
        bodydiff[row]=0.000001
    ratio1[row] = Highdiff[row]/bodydiff[row]
    ratio2[row] = Lowdiff[row]/bodydiff[row]

    if (ratio1[row]>1 and Lowdiff[row]<0.2*Highdiff[row] and bodydiff[row]>bodydiffmin):# and Open[row]>Close[row]):
        return 1
    elif (ratio2[row]>1 and Highdiff[row]<0.2*Lowdiff[row] and bodydiff[row]>bodydiffmin):# and Open[row]<Close[row]):
        return 2
    else:
        return 0
    
def CloseResistance(l,levels,lim):
    if len(levels)==0:
        return 0
    c1 = abs(df.High[l]-min(levels, key=lambda x:abs(x-df.High[l])))<=lim
    c2 = abs(max(df.Open[l],df.Close[l])-min(levels, key=lambda x:abs(x-df.High[l])))<=lim
    c3 = min(df.Open[l],df.Close[l])<min(levels, key=lambda x:abs(x-df.High[l]))
    c4 = df.Low[l]<min(levels, key=lambda x:abs(x-df.High[l]))
    if( (c1 or c2) and c3 and c4 ):
        return 1
    else:
        return 0
    
def CloseSupport(l,levels,lim):
    if len(levels)==0:
        return 0
    c1 = abs(df.Low[l]-min(levels, key=lambda x:abs(x-df.Low[l])))<=lim
    c2 = abs(min(df.Open[l],df.Close[l])-min(levels, key=lambda x:abs(x-df.Low[l])))<=lim
    c3 = max(df.Open[l],df.Close[l])>min(levels, key=lambda x:abs(x-df.Low[l]))
    c4 = df.High[l]>min(levels, key=lambda x:abs(x-df.Low[l]))
    if( (c1 or c2) and c3 and c4 ):
        return 1
    else:
        return 0

In [5]:
n1=1
n2=0
backCandles=30
signal = [0] * length

for row in range(backCandles, len(df)-n2):
    ss = []
    rr = []
    for subrow in range(row-backCandles+n1, row+1):
        if support(df, subrow, n1, n2):
            ss.append(df.Low[subrow])
        if resistance(df, subrow, n1, n2):
            rr.append(df.High[subrow])
    '''#!!!! parameters
    if ((isEngulfing(row)==1 or isStar(row)==1) and CloseResistance(row, rr, 150e-5) ):#and df.RSI[row]<30
        signal[row] = 1
    elif((isEngulfing(row)==2 or isStar(row)==2) and CloseSupport(row, ss, 150e-5)):#and df.RSI[row]>70
        signal[row] = 2
    else:
        signal[row] = 0'''

    if  CloseResistance(row, rr, 150e-5):#and df.RSI[row]<30
        signal[row] = 1
    elif CloseSupport(row, ss, 150e-5):#and df.RSI[row]>70
        signal[row] = 2
    else:
        signal[row] = 0


tempo_lista =10
for num, i in enumerate(signal):
    #print(num,i)
    if i != 0:
        for i in range(tempo_lista):
            try:
                n_num = num + i + 1
                signal[n_num] = 0
            except:
                pass

In [6]:
df['signal']=signal

#df['signal'] = df['signal'].shift(1)

In [7]:
df[df['signal'] != 0].tail(10)

,Data,Open,High,Low,Close,Vol,signal
393,2023-01-13 15:40:00,1.08117,1.08140,1.08034,1.08050,0,1
404,2023-01-13 16:35:00,1.08034,1.08057,1.07973,1.08021,0,1
415,2023-01-13 17:30:00,1.08261,1.08298,1.08177,1.08211,0,1
426,2023-01-13 18:25:00,1.08189,1.08244,1.08164,1.08187,0,1
437,2023-01-13 19:20:00,1.08193,1.08218,1.08184,1.08217,0,1
448,2023-01-13 20:15:00,1.08261,1.08267,1.08238,1.08238,0,1
459,2023-01-13 21:10:00,1.08297,1.08343,1.08297,1.08343,0,1
470,2023-01-13 22:05:00,1.08297,1.08302,1.08274,1.08301,0,1
481,2023-01-13 23:00:00,1.08329,1.08335,1.08301,1.08303,0,1
492,2023-01-16 00:00:00,1.08295,1.08348,1.08294,1.08308,0,1


In [8]:
df[df['signal']==2].count()

Data      2
Open      2
High      2
Low       2
Close     2
Vol       2
signal    2
dtype: int64

In [9]:
def SIGNAL():
    return df.signal

In [10]:
#A new strategy needs to extend Strategy class and override its two abstract methods: init() and next().
#Method init() is invoked before the strategy is run. Within it, one ideally precomputes in efficient, 
#vectorized manner whatever indicators and signals the strategy depends on.
#Method next() is then iteratively called by the Backtest instance, once for each data point (data frame row), 
#simulating the incremental availability of each new full candlestick bar.

#Note, backtesting.py cannot make decisions / trades within candlesticks — any new orders are executed on the
#next candle's Open (or the current candle's Close if trade_on_Close=True). 
#If you find yourself wishing to trade within candlesticks (e.g. daytrading), you instead need to begin 
#with more fine-grained (e.g. hourly) data.

In [11]:
df['Data'] = pd.to_datetime(df['Data'])
df = df.set_index('Data')

In [12]:
df

,Open,High,Low,Close,Vol,signal
Data,,,,,,
2023-01-12 06:55:00,1.07706,1.07736,1.07704,1.07716,0,0
2023-01-12 07:00:00,1.07718,1.07722,1.07705,1.07709,0,0
2023-01-12 07:05:00,1.07709,1.07709,1.07688,1.07698,0,0
2023-01-12 07:10:00,1.07701,1.07727,1.07700,1.07727,0,0
2023-01-12 07:15:00,1.07727,1.07751,1.07712,1.07739,0,0
...,...,...,...,...,...,...
2023-01-16 00:15:00,1.08290,1.08316,1.08287,1.08292,0,0
2023-01-16 00:20:00,1.08292,1.08294,1.08290,1.08294,0,0
2023-01-16 00:25:00,1.08294,1.08294,1.08291,1.08292,0,0


In [14]:
from backtesting import Strategy, Backtest

class MyCandlesStrat(Strategy):  
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next() 
        sl = 0.0010
        tp = 0.0010
        if self.signal1==2:
            sl1 = self.data.Close[-1] - sl
            tp1 = self.data.Close[-1] + tp
            self.buy(sl=sl1, tp=tp1)
        elif self.signal1==1:
            sl1 = self.data.Close[-1] + sl
            tp1 = self.data.Close[-1] - tp
            self.sell(sl=sl1, tp=tp1)

bt = Backtest(df, MyCandlesStrat, cash=100000000, commission=0)
bt.run()


c:\Users\Usuário\AppData\Local\Programs\Python\Python310\lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501


Start                     2023-01-12 06:55:00
End                       2023-01-16 00:35:00
Duration                      3 days 17:40:00
Exposure Time [%]                        62.8
Equity Final [$]               99643993.59574
Equity Peak [$]               100150661.96789
Return [%]                          -0.356006
Buy & Hold Return [%]                0.555164
Return (Ann.) [%]                    8.624669
Volatility (Ann.) [%]                1.156032
Sharpe Ratio                         7.460578
Sortino Ratio                             inf
Calmar Ratio                        12.910185
Max. Drawdown [%]                   -0.668051
Avg. Drawdown [%]                   -0.205353
Max. Drawdown Duration        3 days 13:35:00
Avg. Drawdown Duration        0 days 21:43:00
# Trades                                   35
Win Rate [%]                        45.714286
Best Trade [%]                       0.093874
Worst Trade [%]                     -0.095182
Avg. Trade [%]                    

In [15]:
bt.plot()

Row(id='1418', ...)

In [190]:
#fixed distance Trailing SL
from backtesting import Strategy, Backtest
import numpy as np

class MyCandlesStrat(Strategy):
    sltr=200
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        sltr = self.sltr
        for trade in self.trades: 
            if trade.is_long: 
                trade.sl = max(trade.sl or -np.inf, self.data.Close[-1] - sltr)
            else:
                trade.sl = min(trade.sl or np.inf, self.data.Close[-1] + sltr) 

        if self.signal1==2 and len(self.trades)==0: # trades number change!
            sl1 = self.data.Close[-1] - sltr
            self.buy(sl=sl1)
        elif self.signal1==1 and len(self.trades)==0: # trades number change!
            sl1 = self.data.Close[-1] + sltr
            self.sell(sl=sl1)


bt = Backtest(df, MyCandlesStrat, cash=100000000, commission=0)
stat = bt.run()
stat

C:\Users\Usuário\AppData\Local\Temp\ipykernel_8240\4108643265.py:28: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df, MyCandlesStrat, cash=100000000, commission=0)


Start                                     0.0
End                                    4999.0
Duration                               4999.0
Exposure Time [%]                        40.1
Equity Final [$]                   92889200.0
Equity Peak [$]                   101644655.0
Return [%]                            -7.1108
Buy & Hold Return [%]                1.943471
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -10.755302
Avg. Drawdown [%]                   -3.008073
Max. Drawdown Duration                 4829.0
Avg. Drawdown Duration                 1240.0
# Trades                                401.0
Win Rate [%]                        33.416459
Best Trade [%]                       2.078146
Worst Trade [%]                     -1.427745
Avg. Trade [%]                    

In [27]:
bt.plot(show_legend=False,plot_equity=False)

Row(id='1901', ...)